In [7]:
import torch
import numpy as np
import cupy as cpy
from tqdm import tqdm
import eznf
from eznf import Tensor
from eznf import datasets

In [8]:
class test(eznf.nn.Module):
    def __init__(self):
        super().__init__()
        self.networks = [
            eznf.nn.Cov2d(1, 3, 3),
            eznf.nn.MaxPooling(2),
            eznf.nn.Flatten(),
            eznf.nn.Linear(507, 256),
            eznf.nn.ReLU(),
            eznf.nn.Linear(256, 10)
        ]
    
    def forward(self, x):
        for i in self.networks:
            x = i(x)
        return x

In [9]:
dataset = datasets.MNIST('./', False)
data = dataset.get()

In [10]:
X_train, Y_train, X_test, Y_test = data
X_train = X_train / 255
# X_train = np.vstack([i.flatten() for i in X_train.item])
Y_train = eznf.one_hot(Tensor(Y_train), 10)
X_train = eznf.Tensor(X_train, requires_grad=False)
X_train = X_train[:,None,:,:]

In [11]:
def zero_grad(m):
    for w in m.parameters():
        w.grad = None

def SGD(m: eznf.nn.Module, alpha):
    # 梯度下降
    for w in m.parameters():
        w.item = w.item - alpha*w.grad.item

In [12]:
epoches = 100
m = test()
loss = eznf.nn.CrossEntropyLoss()
batch_size = 1024
steps = len(X_train) // batch_size
with tqdm(total=epoches) as t:
    for i in range(epoches):
        for j in range(steps):
            x = X_train[j*batch_size : (j+1)*batch_size]
            y = Y_train[j*batch_size : (j+1)*batch_size]
            out = m(x)
            l = loss(out, y.T) / batch_size
            l.backward()
            SGD(m, 0.01)
            zero_grad(m)
        
        t.set_description('Epoch {}'.format(i), refresh=False)
        t.set_postfix(loss=l.item[0], refresh=False)
        t.update(1)

  0%|          | 0/100 [00:04<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (10,1024) (10,60000) 